# Projeto 1 - Ciência dos Dados

Nome:Livia Brigido do Nascimento

Nome:Fabio Dotoli Ferreira Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/fabiodff/Desktop/P1Cdados2021_2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [41]:
filename = 'Airpods.xlsx'

In [42]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificação
0,pqp achei que eu tinha perdido a capinha do ai...,0.0
1,@taecgiri que airpods podre é esse que fica ca...,1.0
2,"@vanteanjo army bomb, um álbum do bts (qualque...",0.0
3,g perdu mes airpods pro 😐,0.0
4,não tô acreditando q esqueci meu airpods dentr...,0.0


In [57]:
train.Classificação.value_counts(True)

0.0    0.682274
1.0    0.317726
Name: Classificação, dtype: float64

In [43]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@stevenzmoon @klausbestofall @julietteftthali ...,0.0
1,a minha mãe comprou a uns dias airpods e não m...,0.0
2,"postar uma música de um aplicativo bom, lembre...",0.0
3,se aquele airpods n tivesse identificação do 9...,0.0
4,"sim, comprei airpods com o cartão do bozo hmm ...",0.0


In [56]:
test.Relevância.value_counts(True)

0.0    0.854271
1.0    0.145729
Name: Relevância, dtype: float64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

### Produto escolhido ###
Os Airpods, lançamento da Apple no ano de 2016, são pequenos fones com conexão via bluetooth, que vêm acompanhados do estojo (usado para recarregá-los) e de um cabo lightning (carregador).

- Os tweets considerados relevantes foram aqueles que citavam alguma caracteristica relacionada ao produto escolhido, como por exemplo a durabilidade, a facilidade em perder os fones, preço e críticas/elogios aos Airpods como um todo.

- Os tweets irrelevantes, por sua vez, são aqueles que citam o produto, porém de uma forma que não é possivel extrair um feedback significativo sobre satisfação ou qualidades (positivas ou negativas) como, por exemplo, uma frase citando alguns itens aleatórios (dentre eles, os Airpods).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [52]:
#Montando a função de limpeza (cleanup)

import re #utilizaremos para limpar o texto ('replace')


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text) #substitui o primeiro argumento pelo segundo, e o terceiro argumento é onde será aplicada a substituição
    text_subbed = re.sub('https', "", text_subbed)#vimos que haviam muitos links, e que os mesmos poderiam atrapalhar no desempenho do classificador
    text_subbed = re.sub('\n', '', text_subbed)#retirando o enter (pular para a próxima linha)
    return text_subbed

    

#### Para os tweets Relevantes: ####

In [55]:
#perguntar como retirar os @ e oa nomes de usuários que os acompanham
#perguntar sobre os espaços entre palavras e emojis
#pesquisar sobre a suavização de laplace

In [65]:
#já foi dividido em rel. e irrel. pois depois não será possivel dividir por ser um df de apenas uma coluna 
train_rel_clean = train.loc[train["Classificação"] == 1, 'Treinamento'].apply(cleanup) 
train_rel_clean.to_frame()

,Treinamento
1,@taecgiri que airpods podre é esse que fica ca...
9,@aquelerebelde o airpods é a melhor coisa q eu...
10,perdi um dos airpods em casa e só percebi depo...
12,meus airpods finalmente chegaram
14,meu white people problem de hoje é que necessi...
...,...
277,tô assistindo homem de ferro no airpods e pare...
280,foda do airpods é a porra do sensor deito a ca...
281,ainda n me enviaram o codigo de rastreio dos m...
291,@lovsjatp modo ambiente dos airpodspra vc escu...


In [73]:
train_rel_clean.astype(str)
train_rel_lower = train_rel_clean.str.lower() #por que usar o .str ao inves de uma variavel
train_rel = train_rel_lower.str.split()

1      [@taecgiri, que, airpods, podre, é, esse, que,...
9      [@aquelerebelde, o, airpods, é, a, melhor, coi...
10     [perdi, um, dos, airpods, em, casa, e, só, per...
12                 [meus, airpods, finalmente, chegaram]
14     [meu, white, people, problem, de, hoje, é, que...
                             ...                        
277    [tô, assistindo, homem, de, ferro, no, airpods...
280    [foda, do, airpods, é, a, porra, do, sensor, d...
281    [ainda, n, me, enviaram, o, codigo, de, rastre...
291    [@lovsjatp, modo, ambiente, dos, airpodspra, v...
297                             [perdi, meu, airpods, 🤡]
Name: Treinamento, Length: 95, dtype: object

In [74]:
train_rel_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_rel:
    train_rel_l += palavras
serie_train_rel = pd.Series(train_rel_l)
serie_train_rel

0       @taecgiri
1             que
2         airpods
3           podre
4               é
          ...    
1412         caso
1413        perdi
1414          meu
1415      airpods
1416            🤡
Length: 1417, dtype: object

In [76]:
serie_train_rel_abs = serie_train_rel.value_counts(True)
serie_train_rel_abs

airpods        0.068454
que            0.025406
um             0.023994
o              0.023289
de             0.023289
                 ...   
abençoada      0.000706
desse          0.000706
gg             0.000706
pais           0.000706
sumirammmmm    0.000706
Length: 574, dtype: float64

#### Para os tweets Irrelevantes: ####

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**